In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statistics as stats
from os import listdir
from skimage import io, filters, color,exposure,feature,measure,segmentation
from scipy import ndimage
import cv2 
import imutils
import glob
import sys
sys.path.append('../scripts/')
from nuclei_segmentation_opencv import nuclei_segmenter

In [4]:
df_ext = pd.read_csv('../data/processed/Sipakmed_nuclei_database.csv')

In [5]:
df_ext.head()

,Unnamed: 0,Unnamed: 0.1,cluster_id,image_id,area,major_axis_length,minor_axis_length,eccentricity,orientation,equivalent_diameter,...,meanI_B,meanC_B,smooth_B,moment-3-B,uniformity-B,entropy-B,Nucleus/Cytoplasm,Class,Normal,major_to_minor
0,0,0,1,1,108300.0,394.10,363.96,0.38358,48.3380,371.34,...,105.69,10.8940,0.001822,0.005349,0.027627,5.4535,nuc,s,1,1.082811
1,1,1,1,2,50590.0,330.71,217.29,0.75385,5.8796,253.80,...,115.35,10.2090,0.001600,-0.008656,0.028294,5.3459,nuc,s,1,1.521975
2,2,2,1,3,95258.0,450.82,290.24,0.76518,22.9520,348.26,...,110.24,11.0850,0.001886,0.005759,0.025814,5.4751,nuc,s,1,1.553266
3,3,3,1,4,84199.0,363.33,310.51,0.51924,-14.9310,327.42,...,106.27,11.2150,0.001930,0.009261,0.029336,5.4339,nuc,s,1,1.170107
4,4,4,1,5,98175.0,433.48,303.43,0.71416,-22.9330,353.55,...,115.86,9.7206,0.001451,-0.004682,0.029976,5.2955,nuc,s,1,1.428600


In [6]:
df = df_ext.copy()
df = df[['cluster_id','Class','area','major_axis_length','minor_axis_length','major_to_minor','eccentricity','solidity','Normal']]
df.head()

,cluster_id,Class,area,major_axis_length,minor_axis_length,major_to_minor,eccentricity,solidity,Normal
0,1,s,108300.0,394.10,363.96,1.082811,0.38358,0.94167,1
1,1,s,50590.0,330.71,217.29,1.521975,0.75385,0.88165,1
2,1,s,95258.0,450.82,290.24,1.553266,0.76518,0.90680,1
3,1,s,84199.0,363.33,310.51,1.170107,0.51924,0.91911,1
4,1,s,98175.0,433.48,303.43,1.428600,0.71416,0.92263,1


#### Construct training and testing sets that respect cluster id boundaries: try to minimize data leakage

In [42]:
X = df.drop(['cluster_id','Class','Normal'],axis=1)
y = df['Normal']
clusters = df[['cluster_id','Class']]

# For each class, split each class 70-30 between train and test, respecting clusters (i.e., no shuffling)
# Note that 's' and 'p' are normal classes while 'd' and 'k' are abnormal
# That means that splitting by class will also create an even representation of the target variable

X_train = pd.concat([X[df['Class']==i][df[df['Class']==i].cluster_id.values < np.round(np.percentile(df[df['Class']==i].cluster_id,70))] for i in clusters.Class.unique()])
y_train = pd.concat([y[df['Class']==i][df[df['Class']==i].cluster_id.values < np.round(np.percentile(df[df['Class']==i].cluster_id,70))] for i in clusters.Class.unique()])
X_test = pd.concat([X[df['Class']==i][df[df['Class']==i].cluster_id.values > np.round(np.percentile(df[df['Class']==i].cluster_id,70))] for i in clusters.Class.unique()])
y_test = pd.concat([y[df['Class']==i][df[df['Class']==i].cluster_id.values > np.round(np.percentile(df[df['Class']==i].cluster_id,70))] for i in clusters.Class.unique()])


#### Train the classifier, get performance on the test set

In [125]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,accuracy_score, confusion_matrix, roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve, auc, f1_score
clf = RandomForestClassifier(max_depth=10)
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=10)

In [132]:
# Compare performance on the training set vs the test set: check for over-fitting
trained_preds = clf.predict(X_train)
test_preds = clf.predict(X_test)
print('AUC on the training set:', roc_auc_score(y_train,trained_preds))
print('AUC on the test test:', roc_auc_score(y_test,test_preds))

AUC on the training set: 0.9638476606353408
AUC on the test test: 0.7045174537987681


In [129]:
from sklearn.metrics import classification_report,accuracy_score, confusion_matrix, roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve, auc, f1_score
preds = clf.predict(X_test)
print(f'Classification report: \n{classification_report(y_test,preds)}')
print()
print(f'Confusion matrix: \n{confusion_matrix(y_test,preds)}')
print()
# precision recall curve 
precision, recall, thresholds = precision_recall_curve(y_test,preds)
print(f'Area under the precision-recall curve: \n{auc(recall,precision)}')

Classification report: 
              precision    recall  f1-score   support

           0       0.67      0.81      0.73       487
           1       0.76      0.60      0.67       480

    accuracy                           0.71       967
   macro avg       0.71      0.70      0.70       967
weighted avg       0.71      0.71      0.70       967


Confusion matrix: 
[[394  93]
 [192 288]]

Area under the precision-recall curve: 
0.7772288675911374


#### Try tuning hyperparameters